In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-v0.3", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth: Fast Mistral patching release 2024.6
   \\   /|    GPU: NVIDIA A100-SXM4-80GB. Max memory: 79.151 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",

                      "embed_tokens", "lm_head",], # Add for continual pretraining
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,   # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Offloading input_embeddings to disk to save VRAM
Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2024.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Unsloth: Casting embed_tokens to float32
Unsloth: Casting lm_head to float32


In [3]:
# Wikipedia provides a title and an article text.
# Use https://translate.google.com!
_wikipedia_prompt = """Wikipedia Article
### Title: {}

### Article:
{}"""
# becomes:
wikipedia_prompt = """위키피디아 기사
### 제목: {}

### 기사:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    titles = examples["title"]
    texts  = examples["text"]
    outputs = []
    for title, text in zip(titles, texts):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = wikipedia_prompt.format(title, text) + EOS_TOKEN
        outputs.append(text)
    return { "text" : outputs, }
pass

In [4]:
from datasets import load_dataset

dataset = load_dataset("wikimedia/wikipedia", "20231101.ko", split = "train",)

# We select 1% of the data to make training faster!
dataset = dataset.train_test_split(train_size = 0.01)["train"]

Generating train split:   0%|          | 0/647897 [00:00<?, ? examples/s]

In [5]:
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments

trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,

    args = UnslothTrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,

        # Use warmup_ratio and num_train_epochs for longer runs!
        max_steps = 120,
        warmup_steps = 10,
        # warmup_ratio = 0.1,
        # num_train_epochs = 1,

        # Select a 2 to 10x smaller learning rate for the embedding matrices!
        learning_rate = 5e-5,
        embedding_learning_rate = 1e-5,

        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/6478 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any value given in num_train_epochs
[codecarbon INFO @ 12:59:45] [setup] RAM Tracking...
[codecarbon INFO @ 12:59:45] [setup] GPU Tracking...
[codecarbon INFO @ 12:59:45] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 12:59:45] [setup] CPU Tracking...
[codecarbon WARNING @ 12:59:45] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 12:59:47] CPU Model on constant consumption mode: AMD EPYC 7763 64-Core Processor
[codecarbon INFO @ 12:59:47] >>> Tracker's metadata:
[codecarbon INFO @ 12:59:47]   Platform system: Linux-5.4.0-162-generic-x86_64-with-glibc2.35
[codecarbon INFO @ 12:59:47]   Python version: 3.10.14
[codecarbon INFO @ 12:59:47]   CodeCarbon version: 2.3.5
[codecarbon INFO @ 12:59:47]   Available RAM : 2003.876 GB
[cod

In [6]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 6,478 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 8
\        /    Total batch size = 16 | Total steps = 120
 "-____-"     Number of trainable parameters = 603,979,776


Unsloth: Setting lr = 1.00e-05 instead of 5.00e-05 for embed_tokens.
Unsloth: Setting lr = 1.00e-05 instead of 5.00e-05 for lm_head.


Step,Training Loss
1,1.619100
2,1.612800
3,1.556800
4,1.505100
5,1.649300
6,1.577400
7,1.338600


[codecarbon INFO @ 13:00:33] Energy consumed for RAM : 0.003132 kWh. RAM Power : 751.453685760498 W
[codecarbon INFO @ 13:00:33] Energy consumed for all GPUs : 0.000860 kWh. Total GPU Power : 206.3731578150052 W
[codecarbon INFO @ 13:00:33] Energy consumed for all CPUs : 0.000584 kWh. Total CPU Power : 140.0 W
[codecarbon INFO @ 13:00:33] 0.004576 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:00:48] Energy consumed for RAM : 0.006262 kWh. RAM Power : 751.453685760498 W
[codecarbon INFO @ 13:00:48] Energy consumed for all GPUs : 0.002239 kWh. Total GPU Power : 331.0007781763298 W
[codecarbon INFO @ 13:00:48] Energy consumed for all CPUs : 0.001167 kWh. Total CPU Power : 140.0 W
[codecarbon INFO @ 13:00:48] 0.009668 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:01:03] Energy consumed for RAM : 0.009391 kWh. RAM Power : 751.453685760498 W
[codecarbon INFO @ 13:01:03] Energy consumed for all GPUs : 0.003728 kWh. Total GPU Power : 357.3919068362631 

KeyboardInterrupt: 

[codecarbon INFO @ 13:01:33] Energy consumed for RAM : 0.015620 kWh. RAM Power : 751.453685760498 W
[codecarbon INFO @ 13:01:33] Energy consumed for all GPUs : 0.005613 kWh. Total GPU Power : 94.81615868840137 W
[codecarbon INFO @ 13:01:33] Energy consumed for all CPUs : 0.002914 kWh. Total CPU Power : 140.0 W
[codecarbon INFO @ 13:01:33] 0.024147 kWh of electricity used since the beginning.
